<img src='https://images.contentstack.io/v3/assets/blt57caa63e0368f6e8/blt31e08877cfb6ac56/60708ec3fdb1ac4cff0aaa81/CaptialGroup_Logo_Horizontal.png'>

In [75]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import json
import re
from pprint import pprint
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import sleep
from typing import Any
from selenium import webdriver
from selenium.webdriver.common.by import By
from itertools import chain
from loguru import logger

# Capital Group Insights Class

In [79]:
class CapitalGroupInsights:
    
    def __init__(self):
        
        #urls
        self.equity_url = 'https://www.capitalgroup.com/advisor/insights/categories/equity.html'
        self.fixed_income_url = 'https://www.capitalgroup.com/advisor/insights/categories/fixed-income.html'
        self.retirement_url = 'https://www.capitalgroup.com/advisor/insights/categories/retirement.html'
        self.markets_economy_url = 'https://www.capitalgroup.com/advisor/insights/categories/markets-economy.html'
        
        #options
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("start-maximized")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        webdriver_service = Service('/Users/andishetavakkoli/Documents/chrom_driver/Blackrock/chromedriver')
        self.driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)
        
    
    
    def get_capitalgroup_links(self, main_link: Any) -> list:
        
        self.driver.get(main_link)
        sleep(20)
        equity_link_list = [i.get_attribute('href') for i in self.driver.find_elements(By.CSS_SELECTOR, 'article > a['
                                                                                                          'data-analytics-label'
                                                                                                            '="trackLink:article"]')]
        
        logger.info('all links captured.')
        return equity_link_list

    def capitalgroup_insights(link, article_topic):

        r = requests.get('https://www.capitalgroup.com/advisor/insights/articles/4-reasons-health-care-could-lead-the-next-bull-market.html')
        soup = BeautifulSoup(r.text)
        title = soup.find(class_='cmp-contentfragment--insights__pageTitle').text
        tag = soup.find(class_='cmp-contentfragment--insights__primaryTopic').text
        date = soup.find(class_='cmp-contentfragment--insights__articleDate').text
        raw_content = soup.find(class_='cmp-contentfragment--insights__articlefragment').text
        content = re.sub('\s+',' ' ,raw_content).strip()

        return {
                'index': '',
                'Company': 'Capital Group',
                'topic': 'Insight',
                'article_topic': article_topic,
                'title': title,
                'link': '',
                'date': date,
                'content': content,
                'tag': tag,
            }

    def get_all_article(self, link, article_topic):
        
        link_list = self.get_capitalgroup_links(link)
        article_list = []
        index = 1
        for i in tqdm(link_list):
            article_dict = self.capitalgroup_insights(i)
            article_dict['index'] = index
            article_dict['link'] = i
            article_dict['article_topic'] = article_topic
            article_list.append(article_dict)
            index += 1
            
        logger.info(f'{article_topic} is completed')    
        
        return article_list
    
    
    
    def get_insights(self):
        
        capitalggroup_insight_equity_list = self.get_all_article( self.equity_url, 'Equity')
        capitalggroup_insight_fixed_income_list = self.get_all_article( self.fixed_income_url, 'Fixed_Income')
        capitalggroup_insight_marketeconomy_list = self.get_all_article( self.markets_economy_url, 'Markets&Economy')
        capitalggroup_insight_retirement_list = self.get_all_article( self.retirement_url, 'Retirement')
        
        
        capitalggroup_insight = list(chain(
                                           capitalggroup_insight_equity_list, 
                                           capitalggroup_insight_fixed_income_list,
                                           capitalggroup_insight_marketeconomy_list,
                                           capitalggroup_insight_retirement_list)
                                    )
        logger.info('all articles are integrated.')
        index = 1
        for dict_ in capitalggroup_insight:
            
            dict_['index'] = index
            index += 1
        
        logger.info('Done!')
        
        return capitalggroup_insight                         

In [81]:
obj = CapitalGroupInsights()
capitalgroup_insight = obj.get_insights()

2022-08-25 14:00:18.822 | INFO     | __main__:get_capitalgroup_links:31 - all links captured.
100%|█████████████████████████████████████████| 121/121 [01:03<00:00,  1.90it/s]
2022-08-25 14:01:22.350 | INFO     | __main__:get_all_article:69 - Equity is completed
2022-08-25 14:01:47.527 | INFO     | __main__:get_capitalgroup_links:31 - all links captured.
100%|███████████████████████████████████████████| 50/50 [00:25<00:00,  1.96it/s]
2022-08-25 14:02:13.101 | INFO     | __main__:get_all_article:69 - Fixed_Income is completed
2022-08-25 14:02:40.984 | INFO     | __main__:get_capitalgroup_links:31 - all links captured.
100%|█████████████████████████████████████████| 126/126 [01:03<00:00,  1.99it/s]
2022-08-25 14:03:44.233 | INFO     | __main__:get_all_article:69 - Markets&Economy is completed
2022-08-25 14:04:08.545 | INFO     | __main__:get_capitalgroup_links:31 - all links captured.
100%|███████████████████████████████████████████| 54/54 [00:25<00:00,  2.11it/s]
2022-08-25 14:04:34.168 

In [84]:
capitalgroup_insight

[{'index': 1,
  'Company': 'Capital Group',
  'topic': 'Insight',
  'article_topic': 'Equity',
  'title': '4 reasons health care could lead the next bull market',
  'link': 'https://www.capitalgroup.com/advisor/insights/articles/4-reasons-health-care-could-lead-the-next-bull-market.html',
  'date': 'August 17, 2022',
  'content': 'Amid the storm of market volatility in the first half of 2022, health care was among the few areas that offered investors some shelter. That may not be surprising, considering that demand for services in the sector tend to hold up regardless of market volatility or the economic cycle. In fact, the sector has outpaced the broader global stock market in each of the past eight market declines of 15% or more. Indeed, prospects for select companies appear to be brightening, and some have the potential to remain compelling investment opportunities for years to come, according to equity analyst Christopher Lee, who covers U.S. pharmaceuticals and biotechnology compa

# Save to Json

In [83]:
with open('capitalgroup_insight_dict.json', 'w') as f:
    json.dump(capitalgroup_insight, f, indent=4)

# Equity

In [60]:
equity_link = 'https://www.capitalgroup.com/advisor/insights/categories/equity.html'

In [62]:
capitalggroup_insight_equity = obj.get_all_article(main_link, 'Equity')

/var/folders/rq/68kglm913gz9zwj18qwmq_000000gn/T/ipykernel_32538/3078262074.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/andishetavakkoli/Documents/chrom_driver/chromedriver')
100%|█████████████████████████████████████████| 121/121 [00:56<00:00,  2.14it/s]


In [64]:
len(capitalggroup_insight)

121

# Fix Income

In [65]:
fixedincome_link = 'https://www.capitalgroup.com/advisor/insights/categories/fixed-income.html'

In [66]:
capitalggroup_insight_fixedincome = obj.get_all_article(main_link, 'Fixed_Income')

/var/folders/rq/68kglm913gz9zwj18qwmq_000000gn/T/ipykernel_32538/3078262074.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/andishetavakkoli/Documents/chrom_driver/chromedriver')
100%|█████████████████████████████████████████| 121/121 [00:57<00:00,  2.10it/s]


# Beautiful Soup_ get the content

In [49]:
def capitalgroup_insight(link, article_topic):

    r = requests.get('https://www.capitalgroup.com/advisor/insights/articles/4-reasons-health-care-could-lead-the-next-bull-market.html')
    soup = BeautifulSoup(r.text)
    title = soup.find(class_='cmp-contentfragment--insights__pageTitle').text
    tag = soup.find(class_='cmp-contentfragment--insights__primaryTopic').text
    date = soup.find(class_='cmp-contentfragment--insights__articleDate').text
    raw_content = soup.find(class_='cmp-contentfragment--insights__articlefragment').text
    content = re.sub('\s+',' ' ,raw_content).strip()
    
    return {
            'index': '',
            'Company': 'Capital Group',
            'topic': 'Insight',
            'article_topic': article_topic,
            'title': title,
            'link': link,
            'date': date,
            'content': content,
            'tag': tag,
        }

In [50]:
capitalgroup_insight('https://www.capitalgroup.com/advisor/insights/articles/4-reasons-health-care-could-lead-the-next-bull-market.html' ,'Equity')

{'index': '',
 'Company': 'Capital Group',
 'topic': 'Insight',
 'article_topic': 'Equity',
 'title': '4 reasons health care could lead the next bull market',
 'link': 'https://www.capitalgroup.com/advisor/insights/articles/4-reasons-health-care-could-lead-the-next-bull-market.html',
 'date': 'August 17, 2022',
 'content': 'Amid the storm of market volatility in the first half of 2022, health care was among the few areas that offered investors some shelter. That may not be surprising, considering that demand for services in the sector tend to hold up regardless of market volatility or the economic cycle. In fact, the sector has outpaced the broader global stock market in each of the past eight market declines of 15% or more. Indeed, prospects for select companies appear to be brightening, and some have the potential to remain compelling investment opportunities for years to come, according to equity analyst Christopher Lee, who covers U.S. pharmaceuticals and biotechnology companies. “